In [ ]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [ ]:
# load all models in \cs_mods\July2023 folder by cycling through names
from cobra.io import read_sbml_model    
from pathlib import Path
import pandas as pd

# do not change directory but still go to the folder with the models
folder = Path().cwd() / "cs_mods" / "July2023"
model_list = []
for file in folder.glob("*.xml"):  
    model = read_sbml_model(file)
    model.id = file.stem # remove .xml 
    model.name = file.stem # remove .xml
    model_list.append(model)

In [ ]:
# load old combined RPE_PR models
mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR_old'
mod_RPE_PR.name = 'RPE_PR_old'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod_Human1_Human1.id = 'RPE_PR_control_old'
mod_Human1_Human1.name = 'RPE_PR_control_old'
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')
mod.id = 'Human1_old'
mod.name = 'Human1_old'

# add old models to model_list
model_list = model_list + [mod,mod_RPE_PR,mod_Human1_Human1]

# check out model_list
model_list

In [ ]:
# for all models in the list set objective function (ATP hydrolysis, in PR for combined models)
for m in model_list:
    if 'MAR03964_PR' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_PR'
    elif 'MAR03964_RPE' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_RPE'
    elif 'MAR03964' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964'
    else:
        print('no ATP hydrolysis reaction (MAR03964) in model: ' + m.id)
        

def reset_bounds(model_list):

    # list combined and single models
    combined_models = [m for m in model_list if 'single' not in m.id and 'Human' not in m.id]
    single_models = [m for m in model_list if 'single' in m.id]
    Human1_models = [m for m in model_list if 'Human' in m.id]

    # close PR exchange in combined models and open RPE exchange (efflux)
    for m in combined_models:
        for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
            r.bounds=(0,0)
        for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
            r.bounds=(0,1000) 

    # only allow efflux exchange in single models
    for m in single_models:
        for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
            r.bounds=(0,1000)

    # for Human1 models, open efflux
    for m in [m for m in model_list if 'Human' in m.id]:
        for r in [r for r in m.reactions if len(r.products) == 0]:           # open efflux for RPE exchange reactions
            r.bounds=(0,1000)

reset_bounds(model_list)

In [ ]:
# modify models
# load blood exchange bounds
df_exchange = pd.read_excel(Path().cwd() / 'rxn_bounds/blood_selection.xlsx') # with col names 'id' 'lb' 'ub'
df_exchange

In [ ]:
from src.modify_model import set_exchange_bounds

ex_bounds = {'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(7.25,1000)}
results_dict = {}
for m in model_list:
    set_exchange_bounds(m, ex_bounds)
    m.optimize()
    results_dict[m.id] = m.objective.value
df1 = pd.DataFrame(results_dict, index = ['max ATP hydrolysis (pmol/s/mm^2)']).T
df1

In [ ]:
ex_bounds = {'MAR09048':(-3.51,1000),'MAR09034':(-1,1000),'MAR09135':(0,1000)}
results_dict = {}
for m in model_list:
    set_exchange_bounds(m, ex_bounds)
    m.optimize()
    results_dict[m.id] = m.objective.value
df1 = pd.DataFrame(results_dict, index = ['max ATP hydrolysis (pmol/s/mm^2)']).T
df1

In [ ]:
# import get_met_names
from src.get_info import get_met_names

reset_bounds(model_list)

# set up dicts for results
ovs = {} # objective values
f = {} # fluxes
basis_model = model_list[0] # basis model for metabolite names
df_exchange_ids = list(df_exchange['id_single'])
df_exchange_ids.append('MAR09135') # add lactate

for x in [0,-3.51,-1000]:
    for m in model_list: # for all models in list
        set_exchange_bounds(m,{'MAR09048':(x,1000)}) # set exchange bounds for oxygen 
        for r in df_exchange_ids:
            set_exchange_bounds(m, {r:(-1,1000)}) # set exchange bounds for mets   
            fba = m.optimize() # fba
            ovs[str(m) + ' ' + str(r) + ' o2 = ' + str(x)] = \
            [str(m),str(r),get_met_names(basis_model,r), x, fba.objective_value] # add results to dict
            set_exchange_bounds(m, {r:(0,1000)}) # reset exchange bounds for mets

In [ ]:
# write ovs to df
df_ovs = pd.DataFrame.from_dict(ovs, orient='index', columns = ['model','met_id','met','o2','ATP'])

# single if 'single', 'Human-GEM_28062023', or 'Human1_old' in df1.index else 'combined'
index1= ['single' if 'single' in x \
        or 'Human-GEM_28062023' in x \
        or 'Human1_old' in x \
        else 'combined' for x in list(df_ovs.model)]

# 'Lukowski'if in df1.index, 'Liang' if in df1.index, Human1 if in df1.index else ''
index2 = ['Lukowski' if 'Lukowski' in x \
        else 'Liang' if 'Liang' in x \
        else 'Liang' if 'RPE_PR_old' in x \
        else 'none' if 'Human' or 'control' in x \
        else 'Liang' for x in list(df_ovs.model)]

# 'cone'if ends with cone in df1.index, 'rod' if ends with PR and contains rod in df1.index, RPE if ends with RPE df1.index, Human1 if Human1 in df1.index, else 'RPE-PR
index3 = ['cone' if x.endswith('PR') and 'cone' in x \
        else 'rod' if x.endswith('PR') and 'rod' in x \
        else 'RPE' if x.endswith('RPE') \
        else 'Human1' if 'Human1_old' in x \
        else 'Human1' if 'Human-GEM_28062023' in x \
        else 'Human1-Human1' if 'control'in x \
        else 'RPE-PR rod'  if 'RPE_PR_old'in x
        else 'RPE-PR rod' if 'RPE_PRrod' in x\
        else 'RPE-PR cone' if 'RPE_PRcone' in x\
        else 'RPE-PR rod' for x in list(df_ovs.model)]

index = pd.DataFrame([index1, index2, index3]).T

# add index df to df_ovs vertically without creating nans
df_ovs = pd.concat([df_ovs.reset_index(), index], axis=1, sort=False)
df_ovs = df_ovs.set_index('index')
#rename columns 0,1,2 to 'model_type', 'model', 'cell_type'
df_ovs = df_ovs.rename(columns={0:'single/combined',1:'expression data',2:'model name'})

In [ ]:
df_ovs_single = df_ovs[df_ovs['single/combined'] == 'single']
# pull out unique model names

row_order = set(list(df_ovs_single['model']))

In [ ]:
# make barplot for every single model in df_ov_single

# list models
l=list(df_ovs['model'].unique())

# order bars based on o2 1000 values in Human-GEM_28062023 model
df_ovs_1000_Human = df_ovs[(df_ovs['model'] == 'Human-GEM_28062023') & (df_ovs['o2'] == -1000)] # select df_ovs_1000 for Human-GEM_28062023
df_ovs_1000_Human = df_ovs_1000_Human.sort_values(by = 'ATP', ascending = False) # sort df_ovs_1000_Human by ATP
x_order = list(df_ovs_1000_Human['met']) # get x_order from df_ovs_1000_Human

# pick colour palette
sns.set_palette("rocket", 3)
# make colours of bars lighter
sns.set_style("whitegrid")

for m in l:
    # select part of df_ovs for model m
    df = df_ovs[df_ovs['model'] == m]
    # set up figure
    fig, ax = plt.subplots(figsize=(4, 5))
    # set up barplot
    sns.barplot(x="met", y="ATP", hue="o2", data=df, ax=ax,order=x_order, alpha=0.7)
    # size fig
    fig.set_size_inches(15, 2)
    # set title to model name and expression data from df_ovs
    ax.set_title(df['model name'].unique()[0] + ' ' + df['expression data'].unique()[0])
    # turn x-tick labels
    plt.xticks(rotation=90)
    # y axis label
    plt.ylabel("max ATP hydrolysis \n(pmol/s/mm$^{2}$)")
    # set palette to rocket, make colours very different
    # move legend box out of the way
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # set ylim
    plt.ylim(0, 43)
    # move legend out of figure
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,\
               title="oxygen availability")
    
    # new legend labels
    new_labels = ['unlimited', 'limited (3.51)', 'anaerobic']
    # get legend
    leg = ax.get_legend()
    # change legend labels
    for t, l in zip(leg.texts, new_labels): t.set_text(l)

    # move x axis title up
    ax.xaxis.set_label_coords(0.5, -0.75)

    # make dir for figures
    if not os.path.exists('results/ATP_per_met'):
        os.makedirs('results/ATP_per_met')

    # save figures
    plt.savefig('results/ATP_per_met/' + m + '.png', bbox_inches='tight', dpi=300)

In [ ]:
df_ovs
# turn df_ovs into wide format (melt) with columns 'model', 'met', 'o2', 'ATP'
df_ovs_wide = df_ovs.pivot_table(index=['model','met','single/combined','expression data','model name'], columns='o2', values='ATP')
df_ovs_wide = df_ovs_wide.rename(columns={-3.51:'limited (3.51)',0:'anaerobic',-1000:'unlimited'})
df_ovs_wide.to_clipboard()
# save df_ovs_wide to excel in same folder as figures
df_ovs_wide.to_excel('results/ATP_per_met/df_ovs_wide.xlsx')
df_ovs_wide

In [ ]:
# order bars based on o2 1000 values in Human-GEM_28062023 model
df_ovs_1000_Human = df_ovs[(df_ovs['model'] == 'Human-GEM_28062023') & (df_ovs['o2'] == -1000)] # select df_ovs_1000 for Human-GEM_28062023
df_ovs_1000_Human = df_ovs_1000_Human.sort_values(by = 'ATP', ascending = False) # sort df_ovs_1000_Human by ATP
x_order = list(df_ovs_1000_Human['met']) # get x_order from df_ovs_1000_Human

# from df_ovs only select single models
df_ovs_single = df_ovs[(df_ovs['single/combined'] == 'single') & (df_ovs['model name'] != 'Human1') ]
# catplot df_ovs with o2 as hue and models as rows
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
g = sns.catplot(
    data=df_ovs_single, kind="bar",
    x="met", y="ATP", hue="o2", row="model",
    palette="rocket", alpha=.6, height=2,width=1,
    order = x_order, 
)
# increase bar size
g.despine(left=False)
g.set_axis_labels("","max ATP flux \n(pmol/s/mm$^{2}$)")
g.legend.set_title("")
# rotate x-axis labels
plt.tight_layout()
# rotate x-axis labels
plt.xticks(rotation=90)
# increase width of figure
g.fig.set_figwidth(10)
# Rotate x-axis tick labels for each column
for ax in g.axes.flat:
    plt.setp(ax.get_xticklabels(), rotation=90)


In [ ]:
# order bars based on o2 1000 values in Human-GEM_28062023 model
df_ovs_1000_Human = df_ovs[(df_ovs['model'] == 'Human-GEM_28062023') & (df_ovs['o2'] == -1000)] # select df_ovs_1000 for Human-GEM_28062023
df_ovs_1000_Human = df_ovs_1000_Human.sort_values(by = 'ATP', ascending = False) # sort df_ovs_1000_Human by ATP
x_order = list(df_ovs_1000_Human['met']) # get x_order from df_ovs_1000_Human

# from df_ovs only select single models
df_ovs_single = df_ovs[(df_ovs['single/combined'] == 'single') & (df_ovs['model name'] == 'Human1') ]
# catplot df_ovs with o2 as hue and models as rows
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
g = sns.catplot(
    data=df_ovs_single, kind="bar",
    x="met", y="ATP", hue="o2", row="model",
    palette="rocket", alpha=.6, height=2,width=1,
    order = x_order, 
)
# increase bar size
g.despine(left=False)
g.set_axis_labels("","max ATP flux \n(pmol/s/mm$^{2}$)")
g.legend.set_title("")
# rotate x-axis labels
plt.tight_layout()
# rotate x-axis labels
plt.xticks(rotation=90)
# increase width of figure
g.fig.set_figwidth(10)
# Rotate x-axis tick labels for each column
for ax in g.axes.flat:
    plt.setp(ax.get_xticklabels(), rotation=90)



In [ ]:
# order bars based on o2 1000 values in Human-GEM_28062023 model
df_ovs_1000_Human = df_ovs[(df_ovs['model'] == 'Human-GEM_28062023') & (df_ovs['o2'] == -1000)] # select df_ovs_1000 for Human-GEM_28062023
df_ovs_1000_Human = df_ovs_1000_Human.sort_values(by = 'ATP', ascending = False) # sort df_ovs_1000_Human by ATP
x_order = list(df_ovs_1000_Human['met']) # get x_order from df_ovs_1000_Human

# from df_ovs only select single models
df_ovs_single = df_ovs[(df_ovs['single/combined'] == 'combined') & (df_ovs['model name'] != 'Human1') &(~df_ovs['model'].str.contains('control'))]
# catplot df_ovs with o2 as hue and models as rows
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
g = sns.catplot(
    data=df_ovs_single, kind="bar",
    x="met", y="ATP", hue="o2", row="model",
    palette="rocket", alpha=.6, height=2,width=1,
    order = x_order, 
)
# increase bar size
g.despine(left=False)
g.set_axis_labels("","max ATP flux \n(pmol/s/mm$^{2}$)")
g.legend.set_title("")
# rotate x-axis labels
plt.tight_layout()
# rotate x-axis labels
plt.xticks(rotation=90)
# increase width of figure
g.fig.set_figwidth(10)
# Rotate x-axis tick labels for each column
for ax in g.axes.flat:
    plt.setp(ax.get_xticklabels(), rotation=90)



In [ ]:
# order bars based on o2 1000 values in Human-GEM_28062023 model
df_ovs_1000_Human = df_ovs[(df_ovs['model'] == 'Human-GEM_28062023') & (df_ovs['o2'] == -1000)] # select df_ovs_1000 for Human-GEM_28062023
df_ovs_1000_Human = df_ovs_1000_Human.sort_values(by = 'ATP', ascending = False) # sort df_ovs_1000_Human by ATP
x_order = list(df_ovs_1000_Human['met']) # get x_order from df_ovs_1000_Human

# from df_ovs only select single models
df_ovs_single = df_ovs[(df_ovs['model name'] == 'Human1') | (df_ovs['model'].str.contains('control'))]
# catplot df_ovs with o2 as hue and models as rows
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
g = sns.catplot(
    data=df_ovs_single, kind="bar",
    x="met", y="ATP", hue="o2", row="model",
    palette="rocket", alpha=.6, height=2,width=1,
    order = x_order, 
)
# increase bar size
g.despine(left=False)
g.set_axis_labels("","max ATP flux \n(pmol/s/mm$^{2}$)")
g.legend.set_title("")
# rotate x-axis labels
plt.tight_layout()
# rotate x-axis labels
plt.xticks(rotation=90)
# increase width of figure
g.fig.set_figwidth(10)
# Rotate x-axis tick labels for each column
for ax in g.axes.flat:
    plt.setp(ax.get_xticklabels(), rotation=90)



In [ ]:
reset_bounds(model_list)

# set up dicts for results
ovs = {} # objective values
f = {} # fluxes
uptake = {} # uptake
secretion = {} # secretion

for x in [0,-3.51,-1000]:
    for m in single_models: # for all models in list
        set_exchange_bounds(m,{'MAR09048':(x,1000)}) # set exchange bounds for oxygen 
        for r in list(df['id_single']):
            set_exchange_bounds(m, {r:(-1,1000)}) # set exchange bounds for mets   
            fba = m.optimize() # fba
            ovs[str(m) + ' ' + str(r) + ' o2 = ' + str(x)] = [str(m),str(r), x, fba.objective_value] # add results to dict
            # get uptake and secretion 
            uptake_summary = m.summary().uptake_flux
            
            secretion_summary = m.summary().secretion_flux

            set_exchange_bounds(m, {r:(0,1000)}) # reset exchange bounds for mets

In [ ]:
        mod= single_models[0]
        boundary_dicts=[{'MAR09048':(0,1000)}, {'MAR09048':(-3.51,1000)}, {'MAR09048':(-1000,1000)}]
        rxns= list(df['id_single'])
        boundary_labels= [{'oxygen':'anaerobic'}, {'oxygen':'limited'}, {'oxygen':'unlimited'}]
        results_name='TEST__dark_new_CS'

        # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
        import pandas as pd
        from datetime import datetime
        from src.get_info import make_rxn_df, make_compact_rxn_df, add_compartment2rxn
        from datetime import datetime

        #  create empty dicts
        bounds = dict() # changed model bounds
        conditions = dict()
        ovs = dict() # objective values
        f = dict() # fba fluxes
        uptake = dict() # uptake fluxes
        secretion = dict() # secretion fluxes
        rxn_info = dict()
        boundary_dicts_df= pd.DataFrame()
        boundary_labels_df= pd.DataFrame()

        i = 0
        ii = 0 
    
        mod = m
        
        # run analysis for every dict in list
        for d in boundary_dicts:

            for k in d:   # set bounds for all keys (rxnIDs in dict)
                set_exchange_bounds(d[k]) # set exchange bounds for all keys (mets in dict)
            for r in rxns:
                r = mod.reactions.get_by_id(r)
                m.objective = r.id
                # run analysis for all ATP consumption reactions
                fba = m.optimize() # fba
                label = r.id
                
                rxn_info[i] =  [r.id,r.name,add_compartment2rxn(r)]

                # objective values
                ovs[i] = [m.objective, m.objective.direction, fba.objective_value]
                print(add_compartment2rxn(r))

                if fba.status == 'optimal':
                    try:
                        # write analysis info / results into dicts

                        # uptake / secretion
                        uptake_summary = m.summary().uptake_flux
                        secretion_summary = m.summary().secretion_flux
                        uptake[i] = pd.DataFrame(uptake_summary['flux'])
                        secretion[i] = pd.DataFrame(secretion_summary['flux'])

                        # all fluxes
                        f[label] = fba.to_frame()['fluxes']

                        pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T
                        # info
                        boundary_dicts_df = pd.concat([boundary_dicts_df,pd.DataFrame(pd.DataFrame(boundary_dicts).iloc[ii]).T])
                        boundary_labels_df = pd.concat([boundary_labels_df,pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T])
                    except:
                        continue
                i = i + 1
            ii= ii + 1

    
        # prepare dfs for excel sheet
        model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
                 index=['id','name','compartments','annotation'],columns = ['model'])

        rxn_df = make_rxn_df(m)   
        compact_rxn_df = make_compact_rxn_df(m)  

        met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
                 index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])

        rxn_info_df = pd.DataFrame(rxn_info).T.rename(\
                      columns={0:'objective rxn id',\
                               1:'objective rxn name',\
                               2:'objective reaction'})

        ovs_df = pd.concat([rxn_info_df,\
                            boundary_dicts_df.reset_index(),\
                            boundary_labels_df.reset_index(),\
                            pd.DataFrame(ovs, index=['objective','objective_direction','objective_value']).T]\
                           ,axis=1).drop('index', axis=1)


        # uptake / secretion dfs

        uptake_df = pd.concat(uptake,axis=1)
        secretion_df = pd.concat(secretion,axis=1)

        secretion_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                    for mets in secretion_df.index],\
                                   columns=['metabolite'],index= secretion_df.index)
        uptake_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                    for mets in uptake_df.index],\
                                   columns=['metabolite'],index= uptake_df.index)

        uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)  
        secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)  

        # sort uptake / secretion dfs
        uptake_df = uptake_df.reindex(uptake_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=False).index)
        secretion_df = secretion_df.reindex(secretion_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=True).index)


        # fluxes df
        fluxes_df = pd.concat([pd.DataFrame(f)],keys=["flux"],axis=1)
        fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
        # sort table on absolute flux size 
        fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
        fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
        # select internal fluces only
        fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]

        # date stamp
        datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   

        # write excel file
        with pd.ExcelWriter('FBA_' + results_name + '_' + datestr + '.xlsx') as writer:  
            model_info.to_excel(writer, sheet_name = 'model')
            rxn_df.to_excel(writer, sheet_name = 'reactions')
            met_df.to_excel(writer, sheet_name = 'metabolites')
            ovs_df.to_excel(writer, sheet_name = 'objective_values')  
            uptake_df.to_excel(writer, sheet_name = 'uptake')    
            secretion_df.to_excel(writer, sheet_name = 'secretion')    
            fluxes_df.to_excel(writer, sheet_name = 'fluxes')     


In [ ]:
reset_bounds(model_list)

# set up dicts for results
ovs = {} # objective values
f = {} # fluxes

for x in [0,-3.51,-1000]:
    for m in single_models: # for all models in list
        set_exchange_bounds(m,{'MAR09048':(x,1000)}) # set exchange bounds for oxygen 
        for r in list(df['id_single']):
            set_exchange_bounds(m, {r:(-1,1000)}) # set exchange bounds for mets   
            fba = m.optimize() # fba
            ovs[str(m) + ' ' + str(r) + ' o2 = ' + str(x)] = [str(m),str(r), x, fba.objective_value] # add results to dict
            # get uptake and secretion 
            set_exchange_bounds(m, {r:(0,1000)}) # reset exchange bounds for mets

In [ ]:
uptake = {}
secretion = {}
fluxes ={}    
ovs = {} # objective values


i=1
set_exchange_bounds(m,{'MAR09048':(x,1000)}) # set exchange bounds for oxygen 
set_exchange_bounds(m, {df['id_single'][0]:(-1,1000)}) # set exchange bounds for mets   

fba=m.optimize()
if fba.status == 'optimal':
    try:
        uptake_summary = m.summary().uptake_flux
        secretion_summary = m.summary().secretion_flux
        uptake[i] = pd.DataFrame(uptake_summary['flux'])
        secretion[i] = pd.DataFrame(secretion_summary['flux'])

        # all fluxes
        f[i] = fba.to_frame()['fluxes']
    except:
        print('error in model: ' + m.id + ' for metabolite: ' + r)
uptake

def fba_ATP_rxns(models, boundary_dicts, rxns, boundary_labels=dict(),results_name=''):    
    """
    run fba for all models in list with all boundary dicts in list and all rxns in list

    Parameters
    ----------
    models : list
        list of cobra.Model objects
    boundary_dicts : list
        list of boundary dicts {rxnID:(lb,ub)}
    rxns : list
        list of rxnIDs (strings)
    boundary_labels : dict, optional    
        dictionary with boundary dicts as keys and labels as values, by default dict()
    results_name : str, optional    
        name for excel file, by default ''

    Returns
    -------
    excel file with results
    
    """

    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from src.get_info import make_rxn_df, make_compact_rxn_df, add_compartment2rxn
    from datetime import datetime

    for mod in models:
        print(mod.id) # print model name
        #  create empty dicts

        ovs = dict() # objective values
        f = dict() # fba fluxes
        uptake = dict() # uptake fluxes
        secretion = dict() # secretion fluxes
        rxn_info = dict()
        boundary_dicts_df= pd.DataFrame()
        boundary_labels_df= pd.DataFrame()
        
        i = 0
        ii = 0 
        
        with mod as m:
                
            # run analysis for every dict in list
            for d in boundary_dicts:

                for k in d:   # set bounds for all keys (rxnIDs in dict)
                    m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict
                
                for rxn in rxns:
                    r = m.reactions.get_by_id(rxn)
                    m.objective = r.id
                    # run analysis for all rxns
                    fba = m.optimize() # fba
                    label = r.id
                    
                    rxn_info[i] =  [r.id,r.name,add_compartment2rxn(r)]

                    # objective values
                    ovs[i] = [m.objective, m.objective.direction, fba.objective_value]
                    print(add_compartment2rxn(r))

                    if fba.status == 'optimal':
                        try:
                            # write analysis info / results into dicts

                            # uptake / secretion
                            uptake_summary = m.summary().uptake_flux
                            secretion_summary = m.summary().secretion_flux
                            uptake[i] = pd.DataFrame(uptake_summary['flux'])
                            secretion[i] = pd.DataFrame(secretion_summary['flux'])

                            # all fluxes
                            f[label] = fba.to_frame()['fluxes']

                            pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T
                            # info
                            boundary_dicts_df = pd.concat([boundary_dicts_df,pd.DataFrame(pd.DataFrame(boundary_dicts).iloc[ii]).T])
                            boundary_labels_df = pd.concat([boundary_labels_df,pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T])
                        except UserWarning(...): # if no fluxes, continue
                            continue
                    i = i + 1
                ii= ii + 1

        
        # prepare dfs for excel sheet
        model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
                index=['id','name','compartments','annotation'],columns = ['model'])
        
        rxn_df = make_rxn_df(m)   
        compact_rxn_df = make_compact_rxn_df(m)  
        
        met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
                index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
        
        rxn_info_df = pd.DataFrame(rxn_info).T.rename(\
                    columns={0:'objective rxn id',\
                            1:'objective rxn name',\
                            2:'objective reaction'})
        
        ovs_df = pd.concat([rxn_info_df,\
                            boundary_dicts_df.reset_index(),\
                            boundary_labels_df.reset_index(),\
                            pd.DataFrame(ovs, index=['objective','objective_direction','objective_value']).T]\
                        ,axis=1).drop('index', axis=1)

        
        # uptake / secretion dfs

        uptake_df = pd.concat(uptake,axis=1)
        secretion_df = pd.concat(secretion,axis=1)
        
        secretion_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                    for mets in secretion_df.index],\
                                columns=['metabolite'],index= secretion_df.index)
        uptake_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                    for mets in uptake_df.index],\
                                columns=['metabolite'],index= uptake_df.index)
        
        uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)  
        secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)  
                                    
        # sort uptake / secretion dfs
        uptake_df = uptake_df.reindex(uptake_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=False).index)
        secretion_df = secretion_df.reindex(secretion_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=True).index)

        
        # fluxes df
        fluxes_df = pd.concat([pd.DataFrame(f)],keys=["flux"],axis=1)
        fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
        # sort table on absolute flux size 
        fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
        fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
        # select internal fluces only
        fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
            
        # date stamp
        datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
        
        # write excel file
        with pd.ExcelWriter('FBA_' + results_name + '_' + datestr + '.xlsx') as writer:  
            model_info.to_excel(writer, sheet_name = 'model')
            rxn_df.to_excel(writer, sheet_name = 'reactions')
            met_df.to_excel(writer, sheet_name = 'metabolites')
            ovs_df.to_excel(writer, sheet_name = 'objective_values')  
            uptake_df.to_excel(writer, sheet_name = 'uptake')    
            secretion_df.to_excel(writer, sheet_name = 'secretion')    
            fluxes_df.to_excel(writer, sheet_name = 'fluxes')     
            
    return 

In [ ]:
# ATP yield per metabolite (flux of 1 for each met)
def ATP_yield_analysis_per_aa(models, bounds_dict):
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    results_dict = dict()

    for m in models:
        for m in models:
            rxns = [r for r in m.reactions if len(r.products) == 0]
            for r in rxns:
                r.bounds=(0,1000) # close all exchange reactions
            for index, row in df.iterrows():  
                if row.id_single in [r.id for r in m.reactions]:
                    m.reactions.get_by_id(row['id_single']).bounds = (-1,1000) # open reaction
                    m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
                    fba = m.optimize()
                    results_dict[row['id_single']] = fba.objective_value
                m.reactions.get_by_id(row['id_single']).bounds = (0,1000) # close reaction again
    return results_dict

r1 = ATP_yield_analysis_per_aa(mod,{'MAR09048':(0,1000)}) # anearobic
r2 = ATP_yield_analysis_per_aa(mod,{'MAR09048':(-2.2,1000)}) # limited oxygen
r3 = ATP_yield_analysis_per_aa(mod,{'MAR09048':(-1000,1000)}) # unlimited oxyen

df_results = pd.DataFrame([r1.keys(),r1.values(),r2.values(),r3.values()]).transpose()
df_results.to_clipboard()
df_results

In [ ]:
def fba_ATP_rxns(models, boundary_dicts, rxns, boundary_labels=dict(),results_name=''):    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from src.get_info import make_rxn_df, make_compact_rxn_df, add_compartment2rxn
    from datetime import datetime

    for mod in models:
        print(mod.id) # print model name
        #  create empty dicts
        bounds = dict() # changed model bounds
        conditions = dict()
        ovs = dict() # objective values
        f = dict() # fba fluxes
        uptake = dict() # uptake fluxes
        secretion = dict() # secretion fluxes
        rxn_info = dict()
        boundary_dicts_df= pd.DataFrame()
        boundary_labels_df= pd.DataFrame()
        
        i = 0
        ii = 0 
        
        with mod as m:
                
            # run analysis for every dict in list
            for d in boundary_dicts:

                for k in d:   # set bounds for all keys (rxnIDs in dict)
                    m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict
                
                for rxn in rxns:
                    r = m.reactions.get_by_id(rxn)
                    m.objective = r.id
                    # run analysis for all rxns
                    fba = m.optimize() # fba
                    label = r.id
                    
                    rxn_info[i] =  [r.id,r.name,add_compartment2rxn(r)]

                    # objective values
                    ovs[i] = [m.objective, m.objective.direction, fba.objective_value]
                    print(add_compartment2rxn(r))

                    if fba.status == 'optimal':
                        try:
                            # write analysis info / results into dicts

                            # uptake / secretion
                            uptake_summary = m.summary().uptake_flux
                            secretion_summary = m.summary().secretion_flux
                            uptake[i] = pd.DataFrame(uptake_summary['flux'])
                            secretion[i] = pd.DataFrame(secretion_summary['flux'])

                            # all fluxes
                            f[label] = fba.to_frame()['fluxes']

                            pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T
                            # info
                            boundary_dicts_df = pd.concat([boundary_dicts_df,pd.DataFrame(pd.DataFrame(boundary_dicts).iloc[ii]).T])
                            boundary_labels_df = pd.concat([boundary_labels_df,pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T])
                        except UserWarning(...): # if no fluxes, continue
                            continue
                    i = i + 1
                ii= ii + 1

        
        # prepare dfs for excel sheet
        model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
                index=['id','name','compartments','annotation'],columns = ['model'])
        
        rxn_df = make_rxn_df(m)   
        compact_rxn_df = make_compact_rxn_df(m)  
        
        met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
                index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
        
        rxn_info_df = pd.DataFrame(rxn_info).T.rename(\
                    columns={0:'objective rxn id',\
                            1:'objective rxn name',\
                            2:'objective reaction'})
        
        ovs_df = pd.concat([rxn_info_df,\
                            boundary_dicts_df.reset_index(),\
                            boundary_labels_df.reset_index(),\
                            pd.DataFrame(ovs, index=['objective','objective_direction','objective_value']).T]\
                        ,axis=1).drop('index', axis=1)

        
        # uptake / secretion dfs

        uptake_df = pd.concat(uptake,axis=1)
        secretion_df = pd.concat(secretion,axis=1)
        
        secretion_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                    for mets in secretion_df.index],\
                                columns=['metabolite'],index= secretion_df.index)
        uptake_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                    for mets in uptake_df.index],\
                                columns=['metabolite'],index= uptake_df.index)
        
        uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)  
        secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)  
                                    
        # sort uptake / secretion dfs
        uptake_df = uptake_df.reindex(uptake_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=False).index)
        secretion_df = secretion_df.reindex(secretion_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=True).index)

        
        # fluxes df
        fluxes_df = pd.concat([pd.DataFrame(f)],keys=["flux"],axis=1)
        fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
        # sort table on absolute flux size 
        fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
        fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
        # select internal fluces only
        fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
            
        # date stamp
        datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
        
        # write excel file
        with pd.ExcelWriter('FBA_' + results_name + '_' + datestr + '.xlsx') as writer:  
            model_info.to_excel(writer, sheet_name = 'model')
            rxn_df.to_excel(writer, sheet_name = 'reactions')
            met_df.to_excel(writer, sheet_name = 'metabolites')
            ovs_df.to_excel(writer, sheet_name = 'objective_values')  
            uptake_df.to_excel(writer, sheet_name = 'uptake')    
            secretion_df.to_excel(writer, sheet_name = 'secretion')    
            fluxes_df.to_excel(writer, sheet_name = 'fluxes')     
            
    return 

In [ ]:
from src.analysis import create_permutation_dicts
ox_labels = create_permutation_dicts({'oxygen':['anaerobic', 'limited oxygen (3.51)', 'unlimited oxygen']})
ox_bounds = create_permutation_dicts({'MAR09048_RPE':[(0,0),(-3.51,1000),(-1000,1000)]})
l_rxns = list(df['id'])
dark_new_estimates